In [1]:
import tensorflow as tf
import pickle
import os
import jieba
import pandas as pd
#import jieba.analyse

os.chdir('C://Users/Jingz/OneDrive/CarrierGene/2019.07.10-HPO (done)/NLP_for_onedrive')
#from NLP_function import *
os.chdir('C://Users/Jingz/OneDrive/CarrierGene/2019.07.10-HPO (done)/NLP_for_onedrive/In_Database_cn') 

### Load and clean

In [ ]:
Df = pd.read_csv('C:/Users/Jingz/OneDrive/CarrierGene/2019.07.10-HPO (done)/NLP_for_onedrive/Data_Prepare/Training_data_final.csv', encoding='ANSI')

# remove class without information
Df = Df.fillna('无')
Df = Df[Df.y_cn!='无']
Df = Df[Df.y_cn!='正常']
Df = Df.drop_duplicates(subset='x_cn')

# remove less informative class
#Df = Df[Df.y_cn_count>=10]

# change number
Df['x_cn'] = Df['x_cn'].apply(lambda x : change_number(x,'cn'))

# define X,Y for model learning
X, Y  = Df['x_cn'], Df['y_cn']
x_cut = X.apply(lambda txt: list(jieba.cut(txt))) #jieba tokenization
x_cut.head()

In [3]:
Df = pd.read_csv('C:/Users/Jingz/OneDrive/CarrierGene/2019.07.10-HPO (done)/NLP_for_onedrive/Data_Prepare/Training_data_final.csv', encoding='ANSI')

In [6]:
Df.head()

,index,x_cn,y_cn,tag,tag_class,check_group,change,y_cn_count,HP
0,4534,先天性单侧输精管缺如,1型糖尿病,NaN,NaN,9.0,NaN,4,HP:0100651
1,4535,非梗阻性无精症,1型糖尿病,NaN,NaN,9.0,NaN,4,HP:0100651
2,4536,无精症,1型糖尿病,NaN,NaN,9.0,NaN,4,HP:0100651
3,4537,胰岛素 - 依赖性糖尿病,1型糖尿病,NaN,NaN,9.0,NaN,4,HP:0100651
4,4538,全部卵异常 蜡样透明带 厚而致密 卵周无间隙,Abnormal zona pellucida morphology,NaN,NaN,9.0,NaN,7,HP:0020156


In [7]:
Df['x_cn'][0] = '先天性单侧输精管缺如'
Df['x_cn'][1] = '非梗阻性无精症'
Df['x_cn'][2] = '全部卵异常'

X = Df['x_cn'].iloc[0:3,]
x_cut = X.apply(lambda txt: list(jieba.cut_for_search(txt))) #jieba tokenization
x_cut.head()

<ipython-input-7-54097c66ca53>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Df['x_cn'][0] = '先天性单侧输精管缺如'
<ipython-input-7-54097c66ca53>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Df['x_cn'][1] = '非梗阻性无精症'
<ipython-input-7-54097c66ca53>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Df['x_cn'][2] = '全部卵异常'


0    [先天, 天性, 先天性, 单侧, 输精, 缺如, 输精管, 输精管缺如]
1                  [梗阻, 梗阻性, 无精症, 非梗阻性无精症]
2                              [全部, 卵, 异常]
Name: x_cn, dtype: object

In [8]:
import jieba.posseg as pseg
jieba.load_userdict('C://Users/Jingz/OneDrive/CarrierGene/2019.07.10-HPO (done)/NLP_for_onedrive/hpo_dict_for_jieba.txt') 
result= X.apply(lambda txt: list(pseg.cut(txt)))
result

0    [先天性/n, 单侧/n, 输精管缺如/x]
1               [非梗阻性无精症/x]
2         [全部/n, 卵/n, 异常/d]
Name: x_cn, dtype: object

In [3]:
# Y : One-hot Encoding
Y_onehot, num_classes, lables = onehot(Y) # One-hot Encoding
#print(lables)
#labels = encoder_lab.categories_
#labels = encoder_lab.classes_
lables.to_csv('C://Users/Jingz/OneDrive/CarrierGene/2019.07.10-HPO (en cours)/NLP_for_onedrive/In_Database_cn/lables_nn.csv', header=True, index=False, encoding="ANSI")


#  Split data
from sklearn.model_selection import train_test_split

x_train, x_test, Y_train, Y_test = train_test_split(
    x_cut, Y_onehot, test_size=0.25, random_state=0)
#print('x_train shape:', x_train.shape, 'Y_train shape:', Y_train.shape, 'x_test shape:', x_test.shape, 'Y_test shape:', Y_test.shape)


# X : BOW
from keras.preprocessing import text, sequence

vocab_size  = 1000 
tokenize = text.Tokenizer(num_words = vocab_size, char_level=False)

tokenize.fit_on_texts(x_train) # only fit on train
X_train = tokenize.texts_to_matrix(x_train, mode='tfidf')
X_test  = tokenize.texts_to_matrix(x_test, mode='tfidf')

print('X_train shape:', X_train.shape, 'Y_train shape:', Y_train.shape, 'X_test shape:', X_test.shape, 'Y_test shape:', Y_test.shape)


# Increase sample size (in our case do not change evaluation metrics)

#from imblearn.over_sampling import SMOTE, ADASYN
#smote = SMOTE('minority')

#X_sm, Y_sm = smote.fit_resample(X_train, Y_train)
#print('X_sm shape:', X_sm.shape, 'x_sm shape:', Y_sm.shape)


# save tokenizer
with open('tokenizer_keras_cn_nn.pickle', 'wb') as handle:
    pickle.dump(tokenize, handle, protocol=pickle.HIGHEST_PROTOCOL)  

X_train shape: (3215, 1000) Y_train shape: (3215, 455) X_test shape: (1072, 1000) Y_test shape: (1072, 455)


Using TensorFlow backend.


### Parameter grid for grid search

In [ ]:
param_grid = dict(num_filters=[32, 64, 128],
                  kernel_size=[3, 5, 7],
                  vocab_size=[vocab_size],
                  embedding_dim=[embedding_dim],
                  maxlen=[maxlen])
model = KerasClassifier(build_fn=create_model,
                        epochs=epochs, batch_size=10,
                        verbose=False)
grid = RandomizedSearchCV(estimator=model, param_distributions=param_grid,
                          cv=4, verbose=1, n_iter=5)
grid_result = grid.fit(X_train, y_train)

In [ ]:
# Evaluate testing set
test_accuracy = grid.score(X_test, y_test)

# Save and evaluate results
prompt = input(f'finished {source}; write to file and proceed? [y/n]')
if prompt.lower() not in {'y', 'true', 'yes'}:
    break
with open(output_file, 'a') as f:
    s = ('Running {} data set\nBest Accuracy : '
         '{:.4f}\n{}\nTest Accuracy : {:.4f}\n\n')
    output_string = s.format(
        source,
        grid_result.best_score_,
        grid_result.best_params_,
        test_accuracy)
    print(output_string)
    f.write(output_string)

### fit sequential models

In [14]:
from keras.models import Sequential
from keras import layers
from keras.layers import Dense, Activation, Dropout

# Build the model
model = Sequential()
model.add(Dense(100, input_shape=(vocab_size,), activation='relu')) # first hidden layer has 100 nodes, use relu activation
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax')) # second hidden layer has num_classes nodes, use softmax activation
# sigmoid for binary classification, or softmax for multi-class classification.
model.compile(loss='categorical_crossentropy', # binary_crossentropy; categorical_crossentropy, kullback_leibler_divergence, sparse_categorical_crossentropy (no need of onehot)
              optimizer='adam', # gradient descent
              metrics=['accuracy']) # Compile Keras Model
print(model.summary())

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 100)               100100    
_________________________________________________________________
dropout_6 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 455)               45955     
Total params: 146,055
Trainable params: 146,055
Non-trainable params: 0
_________________________________________________________________
None


In [15]:
history = model.fit(X_train, Y_train,
                    #X_sm, Y_sm,
                    epochs=20,      # One pass through all of the rows in the training dataset
                    verbose=1,        # show training process
                    validation_data=(X_test, Y_test),
                    batch_size= 50)   # One or more samples considered by the model within an epoch before weights are updated

loss, accuracy = model.evaluate(X_train, Y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, Y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))
#plot_history(history)

Train on 3215 samples, validate on 1072 samples
Epoch 1/20
3215/3215 [==============================] - 0s 84us/step - loss: 5.4710 - accuracy: 0.1309 - val_loss: 4.6855 - val_accuracy: 0.2509
Epoch 2/20
3215/3215 [==============================] - 0s 64us/step - loss: 3.9897 - accuracy: 0.3067 - val_loss: 3.8888 - val_accuracy: 0.3554
Epoch 3/20
3215/3215 [==============================] - 0s 61us/step - loss: 3.2517 - accuracy: 0.4093 - val_loss: 3.4334 - val_accuracy: 0.4272
Epoch 4/20
3215/3215 [==============================] - 0s 60us/step - loss: 2.7199 - accuracy: 0.4936 - val_loss: 3.1024 - val_accuracy: 0.4841
Epoch 5/20
3215/3215 [==============================] - 0s 56us/step - loss: 2.3082 - accuracy: 0.5729 - val_loss: 2.8485 - val_accuracy: 0.5159
Epoch 6/20
3215/3215 [==============================] - 0s 62us/step - loss: 1.9625 - accuracy: 0.6348 - val_loss: 2.6341 - val_accuracy: 0.5588
Epoch 7/20
3215/3215 [==============================] - 0s 52us/step - loss: 1.681

In [ ]:
for i in range(10):
    prediction = model.predict(np.array([X_test[i]]))
    predicted_label = pd.DataFrame(lables)[np.argmax(prediction)]
    print('Actual label   :', Y_test[i])
    print('Predicted label:', Y_test[i])
    print("Predicted label: ", predicted_label,"\n")

##### save model

In [16]:
# save model and architecture to single file
model.save("C://Users/Jingz/OneDrive/CarrierGene/2019.07.10-HPO (en cours)/NLP_for_onedrive/In_Database_cn/M_bow_neuralnetwork_cn.h5")
print("Saved model to disk")

Saved model to disk


# Prediction

In [7]:
# load model and tokenizer
model = tf.keras.models.load_model('C://Users/Jingz/OneDrive/CarrierGene/2019.07.10-HPO (en cours)/NLP_for_onedrive/In_Database_cn/M_bow_neuralnetwork_cn.h5') # or Conv1D_model.h5
print('Load model...')
model.summary()

with open('tokenizer_keras_CN_nn.pickle', 'rb') as handle:
    loaded_tokenizer = pickle.load(handle)
    
lables = pd.read_csv('C://Users/Jingz/OneDrive/CarrierGene/2019.07.10-HPO (en cours)/NLP_for_onedrive/In_Database_cn/lables_nn.csv', encoding="ANSI")

Load model...
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 100)               100100    
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 459)               46359     
Total params: 146,459
Trainable params: 146,459
Non-trainable params: 0
_________________________________________________________________


In [8]:
# res = pd.DataFrame({'X':X, 'real_lable': Y})
Df['pred_lable_nn'] = Df['pred_hp_nn'] = Df['same'] = Df.index.values

for l in range(0,len(X)):
    i = X.index.values[l]
    txt = list(jieba.cut(X[i]))
    txt = loaded_tokenizer.texts_to_matrix([txt], mode='tfidf')
    pred = model.predict_classes(txt)
    pred_lab = lables.loc[lables['y_id']==pred[0]]['y_labels'].reset_index(drop=True)[0]
    pred_hp = lables.loc[lables['y_id']==pred[0]]['y_labels'].reset_index(drop=True)[0]
    Df['pred_lable_nn'][i] = pred_lab

    if Df['y_cn'][i]==Df['pred_lable_nn'][i]:
        Df['same'][i] = 1
    else :
        Df['same'][i] = 0

C:\Users\Jingz\anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\Jingz\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\Jingz\anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a

In [9]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(Df.y_cn, Df.pred_lable_nn)
print('Accuracy: %f' % accuracy)  # same 
# precision tp / (tp + fp)
precision = precision_score(Df.y_cn, Df.pred_lable_nn, average='weighted') # take into account for label imbalance
print('Precision_macro: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(Df.y_cn, Df.pred_lable_nn, average='weighted')
print('Recall_macro: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(Df.y_cn, Df.pred_lable_nn, average='weighted')
print('F1 score_macro: %f' % f1)

Accuracy: 0.880597
Precision_macro: 0.903785
Recall_macro: 0.880597
F1 score_macro: 0.881450


C:\Users\Jingz\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [143]:
Df.to_csv('C://Users/Jingz/OneDrive/CarrierGene/2019.07.10-HPO (en cours)/NLP_for_onedrive/Run/prediction_nn.csv', header=True, index=False, encoding="ANSI")